In [2]:
import gym
import gym_sokoban
import time
import datetime as dt
import matplotlib.pyplot as plt
import math
from collections import deque
import random
import numpy as np
import pickle
import sys
import tqdm
import os
from collections import namedtuple,defaultdict,deque 

In [3]:
chapter = 0
level = 0


env_name = 'Sokoban-v1'
env = gym.make(env_name)
ACTION_LOOKUP = env.unwrapped.get_action_lookup()
env.unwrapped.set_level(chapter,level)
env.seed(0)
env.reset()
print("Created environment: {}".format(env_name))

folder = 'Results/Chapter '+str(chapter)+'/Level '+str(level)
#create Results/Chapter [chapter]/Level [level] folder
if not os.path.exists(folder):
    print("No data found for this level")
#loop and find the latest file with 'P_' in the name, prefer the file with not 'temp' in the name
else:
    files = os.listdir(folder)
    files = [f for f in files if 'P_' in f]
    files = [f for f in files if 'temp' not in f]
    if len(files) == 0:
        print("No data found for this level")
    else:
        files.sort(key=lambda x: os.path.getmtime(os.path.join(folder, x)))
        file = files[-1]
        print("Loading file: {}".format(file))
        with open(folder+'/'+file,'rb') as f:
            P = pickle.load(f)
        print("Loaded file: {}".format(file))



Created environment: Sokoban-v1
Loading file: P_first_100_episodes.bin
Loaded file: P_first_100_episodes.bin


#### Value iteration

In [27]:
def policy_pi(P,s):
    #.9 prob of greedy action
    #.1 prob of random action
    prob = []
    for a in P[s]:
        prob.append(P[s][a][1])
    prob = np.array(prob)
    print(f'\r{prob}',end='                             ')
    return np.random.choice(list(P[s].keys()),p=prob)

for i in range(1000):
    time.sleep(.01)
    _ = env.reset()
    state = env.unwrapped.serialize_state()
    st_time = time.time()
    done = False
    t= 0
    last_reward = 0
    last_time = time.time()
    last_state = state
    rendered_frame = 0
    while True:
        if time.time() - st_time < .3:
            env.render()
            rendered_frame += 1
            time.sleep(1/64)
        st_time = time.time()
        action_time = time.time()
        action = policy_pi(P,state)
        
        state, reward, done, info = env.step(action)
        state = env.unwrapped.serialize_state()
        #print(f'{ACTION_LOOKUP[action]} state change: {last_state != state}',end=' ')
        last_state = state
        last_reward = reward
        t += 1
        if done or t > 150:
            env.render()
            break
        env.render()

[0.00464828 0.         0.55361469 0.17148183 0.27025521]                             

Exception ignored in: <function Texture.__del__ at 0x000001F24B80D550>
Traceback (most recent call last):
  File "d:\2023-2024\RNN\Sokoban\SokobanEnv\lib\site-packages\pyglet\image\__init__.py", line 1225, in __del__
    self._context.delete_texture(self.id)
  File "d:\2023-2024\RNN\Sokoban\SokobanEnv\lib\site-packages\pyglet\gl\base.py", line 321, in delete_texture
    gl.glDeleteTextures(1, gl.GLuint(texture_id))
  File "d:\2023-2024\RNN\Sokoban\SokobanEnv\lib\site-packages\pyglet\gl\lib.py", line 52, in errcheck
    def errcheck(result, func, arguments):
KeyboardInterrupt: 


[0.07611979 0.05442995 0.         0.79665105 0.07279921]                             

In [5]:
def greedy_policy_pi(P,s):
    #.9 prob of greedy action
    #.1 prob of random action
    r_choice = .2
    if np.random.random() < r_choice:
        return np.random.choice(np.arange(env.action_space.n))
    else:
        prob = []
        for a in P[s]:
            prob.append(P[s][a][1])
        prob = np.array(prob)
        print(f'\r{prob}',end='                             ')
        return np.random.choice(list(P[s].keys()),p=prob)

for i in range(1000):
    time.sleep(.1)
    _ = env.reset()
    state = env.unwrapped.serialize_state()
    st_time = time.time()
    done = False
    t= 0
    last_reward = 0
    last_time = time.time()
    last_state = state
    rendered_frame = 0
    while True:
        if time.time() - st_time < .2:
            env.render()
            rendered_frame += 1
            time.sleep(1/64)
            if (time.time() - last_time) > 1:
                print(f'\rFPS: {rendered_frame}',end='')
                rendered_frame = 0
                last_time = time.time()
            continue
        st_time = time.time()
        action_time = time.time()
        action = greedy_policy_pi(P,state)
        
        state, reward, done, info = env.step(action)
        state = env.unwrapped.serialize_state()
        #print(f'{ACTION_LOOKUP[action]} state change: {last_state != state}',end=' ')
        last_state = state
        last_reward = reward
        t += 1
        if done or t > 150:
            env.render()
            break
        env.render()

FPS: 29666  0.16027528 0.17306165 0.         0.48089647]                             

KeyboardInterrupt: 